# ***BINARY BLINKING CLASSIFICATION USING CEW DATASET***

# Import required libraries

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras import layers, models

# Defining hyperparameters and loading the dataset

In [ ]:
# Define the paths to the train and validation data folders
train_data_dir = '/content/drive/MyDrive/CEW/dataset/train'
validation_data_dir = '/content/drive/MyDrive/CEW/dataset/val'

# Set the input image dimensions
img_width, img_height = 24, 24

# Define the batch size and number of training epochs
batch_size = 32
epochs = 10

# Data Preprocessing

In [ ]:
# Data augmentation and normalization for the training set
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Normalization for the validation set
validation_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Load and preprocess the training set
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True
)

# Load and preprocess the validation set
validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

Found 3780 images belonging to 2 classes.
Found 1068 images belonging to 2 classes.


# Loading and Training the Model

In [ ]:
# Define the CNN model architecture
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))  # Binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

Epoch 1/10
118/118 [==============================] - 1655s 14s/step - loss: 0.5648 - accuracy: 0.6942 - val_loss: 0.4376 - val_accuracy: 0.7898
Epoch 2/10
118/118 [==============================] - 11s 90ms/step - loss: 0.3200 - accuracy: 0.8741 - val_loss: 0.2607 - val_accuracy: 0.9091
Epoch 3/10
118/118 [==============================] - 10s 87ms/step - loss: 0.2247 - accuracy: 0.9165 - val_loss: 0.2155 - val_accuracy: 0.9223
Epoch 4/10
118/118 [==============================] - 10s 86ms/step - loss: 0.1842 - accuracy: 0.9304 - val_loss: 0.1743 - val_accuracy: 0.9432
Epoch 5/10
118/118 [==============================] - 11s 97ms/step - loss: 0.1715 - accuracy: 0.9400 - val_loss: 0.1736 - val_accuracy: 0.9318
Epoch 6/10
118/118 [==============================] - 10s 88ms/step - loss: 0.1582 - accuracy: 0.9392 - val_loss: 0.1504 - val_accuracy: 0.9489
Epoch 7/10
118/118 [==============================] - 10s 86ms/step - loss: 0.1477 - accuracy: 0.9453 - val_loss: 0.1541 - val_accuracy

# Evaluating the Model

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation Loss: {loss:.4f}')
print(f'Validation Accuracy: {accuracy:.4f}')

34/34 [==============================] - 2s 58ms/step - loss: 0.1313 - accuracy: 0.9476
Validation Loss: 0.1313
Validation Accuracy: 0.9476


Metrics results

In [ ]:
# Calculate precision, recall, and F1 score
y_true = validation_generator.classes
y_pred = model.predict(validation_generator)
y_pred = (y_pred > 0.5).astype(int)

precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

34/34 [==============================] - 2s 56ms/step
Precision: 0.9310
Recall: 0.9687
F1 Score: 0.9495


Predictions

In [ ]:

# /content/drive/MyDrive/CEW/closedeye.jpg
# /content/drive/MyDrive/CEW/openeye.jpg

from PIL import Image
# Load and preprocess the new eye image
new_eye_image_path = '/content/drive/MyDrive/CEW/openeye.jpg'
new_eye_image = Image.open(new_eye_image_path)
new_eye_image = new_eye_image.resize((img_width, img_height))
new_eye_image = np.array(new_eye_image)
new_eye_image = new_eye_image.astype('float32') / 255.0
# Make predictions on new eye images
prediction = model.predict(np.expand_dims(new_eye_image, axis=0))
if prediction > 0.5:
    print("Blinking")
else:
    print("Not blinking")

1/1 [==============================] - 0s 32ms/step
Blinking
